In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import re
from transformers import AutoTokenizer, DistilBertTokenizer, DistilBertForSequenceClassification, AutoModelForSequenceClassification, Trainer, TrainingArguments, DistilBertConfig
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from nltk.corpus import wordnet
import random
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.nn import CrossEntropyLoss
import nltk
from transformers import DistilBertTokenizer
from sklearn.metrics import classification_report
from huggingface_hub import create_repo
import os
from git import Repo  # For interacting with Git
from huggingface_hub import HfApi
import shutil

In [ ]:
!cd "/Users/adekunleajibode/Documents/GH_repo/distilbert-base-uncased-not-usable-code" && git init
!cd "/Users/adekunleajibode/Documents/GH_repo/distilbert-base-uncased-not-usable-code" && git add .

!cd "/Users/adekunleajibode/Documents/GH_repo/distilbert-base-uncased-not-usable-code" && git commit -m "Initial commit"
!cd "/Users/adekunleajibode/Documents/GH_repo/distilbert-base-uncased-not-usable-code" && git remote add origin https://github.com/eyinlojuoluwa/distilbert-base-uncased-not-usable-code.git
!cd "/Users/adekunleajibode/Documents/GH_repo/distilbert-base-uncased-not-usable-code" && git push -u origin main

In [1]:
import json
import os
import shutil
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from huggingface_hub import create_repo, login, whoami
import os
import shutil
from git import Repo
from git.exc import GitCommandError


with open("tokens.json", "r") as f:
    tokens = json.load(f)

HUGGINGFACE_TOKEN = tokens["HUGGINGFACE_TOKEN"]
GITHUB_TOKEN = tokens["GITHUB_TOKEN"]

# --- Configuration ---
MODEL_DIR = "./final_model"
GITHUB_REPO_PATH = "/Users/adekunleajibode/Documents/GH_repo/distilbert-base-uncased-not-usable-code"
HUGGINGFACE_REPO_NAME = "distilbert-base-uncased-not-usable"
GITHUB_COMMIT_MESSAGE = "Auto-push: Updated notebooks"
HUGGINGFACE_COMMIT_MESSAGE = "Upload fine-tuned model"

# --- Hugging Face Auth + Upload ---
login(token=HUGGINGFACE_TOKEN)
print(f"Logged in as: {whoami()['name']}")
create_repo(HUGGINGFACE_REPO_NAME, exist_ok=True)

model = DistilBertForSequenceClassification.from_pretrained(MODEL_DIR)
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_DIR)

model.push_to_hub(HUGGINGFACE_REPO_NAME, commit_message=HUGGINGFACE_COMMIT_MESSAGE)
tokenizer.push_to_hub(HUGGINGFACE_REPO_NAME, commit_message=HUGGINGFACE_COMMIT_MESSAGE)
print("✅ Model and tokenizer pushed to Hugging Face.")

# --- GitHub Notebook File Sync ---
# Get current directory notebook files
current_notebooks = [f for f in os.listdir() if f.endswith(".ipynb")]

# Replace files in GitHub repo
for nb in current_notebooks:
    src = os.path.join(os.getcwd(), nb)
    dst = os.path.join(GITHUB_REPO_PATH, nb)
    if os.path.exists(dst):
        os.remove(dst)
        print(f"Removed existing file: {dst}")
    shutil.copy(src, dst)
    print(f"Copied {nb} to GitHub repository.")

# --- GitHub Commit & Push ---
os.chdir(GITHUB_REPO_PATH)
os.system("git init")
os.system("git add .")
os.system(f'git commit -m "{GITHUB_COMMIT_MESSAGE}"')
os.system("git branch -M main")
os.system("git remote remove origin || true")
os.system("git remote add origin https://github.com/eyinlojuoluwa/distilbert-base-uncased-not-usable-code.git")
os.system("git push -u origin main")
print("✅ Code pushed to GitHub.")

Logged in as: eyinlojuoluwa


No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


✅ Model and tokenizer pushed to Hugging Face.
Removed existing file: /Users/adekunleajibode/Documents/GH_repo/distilbert-base-uncased-not-usable-code/sync_tool.ipynb
Copied sync_tool.ipynb to GitHub repository.
Removed existing file: /Users/adekunleajibode/Documents/GH_repo/distilbert-base-uncased-not-usable-code/Untitled1.ipynb
Copied Untitled1.ipynb to GitHub repository.
Removed existing file: /Users/adekunleajibode/Documents/GH_repo/distilbert-base-uncased-not-usable-code/learning.ipynb
Copied learning.ipynb to GitHub repository.
Removed existing file: /Users/adekunleajibode/Documents/GH_repo/distilbert-base-uncased-not-usable-code/Untitled.ipynb
Copied Untitled.ipynb to GitHub repository.
Removed existing file: /Users/adekunleajibode/Documents/GH_repo/distilbert-base-uncased-not-usable-code/labels.ipynb
Copied labels.ipynb to GitHub repository.
Removed existing file: /Users/adekunleajibode/Documents/GH_repo/distilbert-base-uncased-not-usable-code/Finetune.ipynb
Copied Finetune.ipyn

remote: error: GH013: Repository rule violations found for refs/heads/main.        
remote: 
remote: - GITHUB PUSH PROTECTION        
remote:   —————————————————————————————————————————        
remote:     Resolve the following violations before pushing again        
remote: 
remote:     - Push cannot contain secrets        
remote: 
remote:             
remote:      (?) Learn how to resolve a blocked push        
remote:      https://docs.github.com/code-security/secret-scanning/working-with-secret-scanning-and-push-protection/working-with-push-protection-from-the-command-line#resolving-a-blocked-push        
remote:             
remote:             
remote:       —— Hugging Face User Access Token ————————————————————        
remote:        locations:        
remote:          - commit: 70ed73b3d4582737f7981788e1f2e62c9bb3423e        
remote:            path: sync_tool.ipynb:47        
remote:          - commit: f97be82bdeaa57585e1e2bb6ef828a2c4cf999cc        
remote:            path: 

In [ ]:
import os
print(os.getcwd())

In [ ]:
ls /Users/your_username/Documents/GH_repo/distilbert-base-uncased-not-usable-code/.git

In [ ]:
import os

# Define the path
repo_path = "/Users/adekunleajibode/Documents/GH_repo/distilbert-base-uncased-not-usable-code"

# List files in the directory
files = os.listdir(repo_path)
print(files)

In [ ]:
repo_name = "distilbert-base-uncased-not-usable"  # Change this to your preferred repository name
create_repo(repo_name)

# Replace these with your actual paths
model_dir = "./final_model"

# Push model, tokenizer, and config to Hugging Face Hub
model = DistilBertForSequenceClassification.from_pretrained(model_dir)
tokenizer = DistilBertTokenizer.from_pretrained(model_dir)

model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)